In [0]:
import json
import os
import sys
import tensorflow as tf

In [0]:
rm -rf {"/content/ll_utils"}

In [8]:
if 'google.colab' in sys.modules:
    !git clone https://github.com/leighlin0511/ll_utils
    sys.path.append('/content/ll_utils')
    
    from google.colab import auth
    auth.authenticate_user()    

import ll_cellular.main_ll as main_ll

Cloning into 'll_utils'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 35 (delta 7), reused 35 (delta 7), pack-reused 0
Unpacking objects: 100% (35/35), done.


## Train

Set `MODEL_DIR` to be a Google Cloud Storage bucket that you can write to.   The code will write your checkpoins to this directory.

In [16]:
MODEL_DIR = 'gs://kaggle-recursive-cell-image-storage/saved_model/1562302335/'
URL_BASE_PATH = 'gs://rxrx1-us-central1/tfrecords/random-42'

# make sure we're in a TPU runtime
assert 'COLAB_TPU_ADDR' in os.environ

# set TPU-relevant args
tpu_grpc = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
num_shards = 8  # colab uses Cloud TPU v2-8

# upload credentials to the TPU
with tf.Session(tpu_grpc) as sess:
    data = json.load(open('/content/adc.json'))
    tf.contrib.cloud.configure_gcs(sess, credentials=data)

tf.logging.set_verbosity(tf.logging.INFO)

#print(main_ll)
from tensorflow.python.estimator import estimator

current_step = estimator._load_global_step_from_checkpoint_dir(MODEL_DIR)
print(current_step)

main_ll.main(use_tpu=True,
     tpu=tpu_grpc,
     gcp_project=None,
     tpu_zone=None,
     url_base_path=URL_BASE_PATH,
     use_cache=False,
     model_dir=MODEL_DIR,
     train_epochs=1,
     train_batch_size=512,
     num_train_images=73030,
     epochs_per_loop=1,
     log_step_count_epochs=1,
     num_cores=num_shards,
     data_format='channels_last',
     transpose_input=True,
     tf_precision='bfloat16',
     n_classes=1108,
     momentum=0.9,
     weight_decay=1e-4,
     base_learning_rate=0.2,
     warmup_epochs=5)

W0711 06:55:56.734894 140059474896768 deprecation_wrapper.py:119] From /content/ll_utils/ll_cellular/main_ll.py:280: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

I0711 06:55:56.736506 140059474896768 main_ll.py:280] tpu: grpc://10.58.188.58:8470
I0711 06:55:56.739896 140059474896768 main_ll.py:283] gcp_project: None


<module 'll_cellular.main_ll' from '/content/ll_utils/ll_cellular/main_ll.py'>
0


W0711 06:55:57.858453 140059474896768 estimator.py:1984] Estimator's model_fn (functools.partial(<function resnet_model_fn at 0x7f61de551e18>, n_classes=1108, num_train_images=73030, data_format='channels_last', transpose_input=True, train_batch_size=512, iterations_per_loop=142, tf_precision='bfloat16', momentum=0.9, weight_decay=0.0001, base_learning_rate=0.2, warmup_epochs=5, model_dir='gs://kaggle-recursive-cell-image-storage/saved_model/1562302335/', use_tpu=True, resnet_depth=50)) includes params argument, but params are not passed to Estimator.
I0711 06:55:57.861337 140059474896768 estimator.py:209] Using config: {'_model_dir': 'gs://kaggle-recursive-cell-image-storage/saved_model/1562302335/', '_tf_random_seed': None, '_save_summary_steps': 142, '_save_checkpoints_steps': 142, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.58.188.58:8470"
    }
  }
}
isolate_session